In [19]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import pickle

In [20]:
csv_filename = "cubes_info.csv"
data = pd.read_csv(csv_filename, header=None, sep=';')  
print(data.dtypes)
print(data.isna().sum())

data.rename(columns={data.columns[0]: 'Target'}, inplace=True)
for i in range(1 , len(data.columns)):
    data.rename(columns={data.columns[i]: f'Features{i}'}, inplace=True)


for col in data.columns:
    if col != 'Target':  
        if data[col].dtype == 'object':
            try:
                data[col] = data[col].apply(lambda x: float(x.split(',')[2]))  # Extraire la pente (slope)
            except Exception as e:
                print(f"Erreur lors de la conversion de la colonne {col}: {e}")
    else:
        print(f"Colonne 'Target' ignorée pour la conversion.")


print(data.dtypes)

X = data.drop(columns=('Target') , axis=1)
Y = data['Target']
scaler = StandardScaler()
X_scaled =  scaler.fit_transform(X)
print('done')

0     object
1     object
2     object
3     object
4     object
       ...  
60    object
61    object
62    object
63    object
64    object
Length: 65, dtype: object
0     0
1     0
2     0
3     0
4     0
     ..
60    0
61    0
62    0
63    0
64    0
Length: 65, dtype: int64
Colonne 'Target' ignorée pour la conversion.
Target         object
Features1     float64
Features2     float64
Features3     float64
Features4     float64
               ...   
Features60    float64
Features61    float64
Features62    float64
Features63    float64
Features64    float64
Length: 65, dtype: object
done


In [21]:
print(X.shape)
print((Y.unique()))

(300, 64)
['b' 'f' 'g' 'h' 'j' 'k']


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2021)


In [ ]:
param_grid = {
    'hidden_layer_sizes': [(64,), (64, 32), (128, 64, 32)], 
    'activation': ['relu', 'tanh'],  
    'solver': ['adam', 'sgd', 'lbfgs'],  
    'alpha': [0.0001, 0.001], 
    'learning_rate': ['constant', 'adaptive'],  
    'max_iter': [500, 1000, 1500] 
}


mlp = MLPClassifier(random_state=2021)

grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print(f"Meilleurs paramètres : {grid_search.best_params_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("/nRapport de classification :/n", classification_report(y_test, y_pred, target_names=[f"Classe {i}" for i in range(6)]))


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Meilleurs paramètres : {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (128, 64, 32), 'learning_rate': 'constant', 'max_iter': 500, 'solver': 'adam'}

Rapport de classification :
               precision    recall  f1-score   support

    Classe 0       0.84      0.89      0.86        18
    Classe 1       0.94      1.00      0.97        15
    Classe 2       1.00      0.81      0.90        16
    Classe 3       0.44      0.64      0.52        11
    Classe 4       0.93      1.00      0.96        13
    Classe 5       0.83      0.59      0.69        17

    accuracy                           0.82        90
   macro avg       0.83      0.82      0.82        90
weighted avg       0.85      0.82      0.83        90



In [23]:


mlp = MLPClassifier(hidden_layer_sizes=(32,), activation='relu', solver='adam', max_iter=1000, random_state=42)

mlp.fit(X_train, y_train)

train_score = mlp.score(X_train, y_train)
test_score = mlp.score(X_test, y_test)

y_pred = mlp.predict(X_test)

report = classification_report(y_test, y_pred, target_names=[f"Classe {i}" for i in range(6)])

print(report)

              precision    recall  f1-score   support

    Classe 0       0.81      0.94      0.87        18
    Classe 1       0.93      0.87      0.90        15
    Classe 2       1.00      0.94      0.97        16
    Classe 3       0.57      0.73      0.64        11
    Classe 4       0.93      1.00      0.96        13
    Classe 5       0.92      0.65      0.76        17

    accuracy                           0.86        90
   macro avg       0.86      0.85      0.85        90
weighted avg       0.87      0.86      0.86        90



In [24]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

metrics = {
    "train_score": round(best_model.score(X_train, y_train),2),
    "test_score":round(best_model.score(X_test, y_test),2),
    "classification_report": classification_report(y_test, y_pred, target_names=[f"Classe {i}" for i in range(6)], output_dict=True),
}

print(metrics)

{'train_score': 1.0, 'test_score': 0.82, 'classification_report': {'Classe 0': {'precision': 0.8421052631578947, 'recall': 0.8888888888888888, 'f1-score': 0.8648648648648649, 'support': 18.0}, 'Classe 1': {'precision': 0.9375, 'recall': 1.0, 'f1-score': 0.967741935483871, 'support': 15.0}, 'Classe 2': {'precision': 1.0, 'recall': 0.8125, 'f1-score': 0.896551724137931, 'support': 16.0}, 'Classe 3': {'precision': 0.4375, 'recall': 0.6363636363636364, 'f1-score': 0.5185185185185185, 'support': 11.0}, 'Classe 4': {'precision': 0.9285714285714286, 'recall': 1.0, 'f1-score': 0.9629629629629629, 'support': 13.0}, 'Classe 5': {'precision': 0.8333333333333334, 'recall': 0.5882352941176471, 'f1-score': 0.6896551724137931, 'support': 17.0}, 'accuracy': 0.8222222222222222, 'macro avg': {'precision': 0.8298350041771094, 'recall': 0.8209979698950286, 'f1-score': 0.816715863063657, 'support': 90.0}, 'weighted avg': {'precision': 0.8474554441659704, 'recall': 0.8222222222222222, 'f1-score': 0.82638760

In [14]:
with open("best_mlp_model.pkl", "wb") as file:
    pickle.dump("model": best_model, file)

print("model.pkl'")

model.pkl'


In [26]:
import pickle

try:
    with open("C:/Users/huetb/Desktop/Cours/Bachelor 3/Projet Lettre ML/code/final/projetml 1/test_model.pkl", "rb") as file:
        loaded_object = pickle.load(file)
    
    print(f"Clés disponibles : {loaded_object.keys()}")
    
    model = loaded_object["model"]
    print(f"Modèle chargé : {type(model)}")
    
    # Test d'une prédiction si les données sont disponibles
    # test_data = ...  # Remplacez par vos données de test
    # prediction = model.predict(test_data)
    # print(f"Prédiction : {prediction}")

except FileNotFoundError:
    print("Erreur : Le fichier .pkl n'existe pas au chemin spécifié.")
except KeyError as e:
    print(f"Erreur : Clé manquante dans le fichier .pkl ({e}).")
except Exception as e:
    print(f"Erreur inattendue : {e}")


Erreur : Le fichier .pkl n'existe pas au chemin spécifié.
